À partir des datasets `population_communes.csv` et `surface_departements.csv`, créer un nouveau dataset qui contient une ligne par département, avec ces colonnes:

- la somme des "Population municipale" du département
- la somme des "Population totale" du département (pour l'explication de la distinction entre "Population municipale" et "Population totale", voir: https://www.insee.fr/fr/metadonnees/definition/c1270)
- la part (en pourcentage) de la population municipale par rapport à la population totale
- la part (en pourcentage) de la population (municipale) du département au sein de sa région
- la densité de la population (municipale) en nb d'habitants / km2

(Le dataset final devrait ressembler à `result-exo-cc.csv`)


In [36]:
import pandas as pd

### Load data

In [37]:
df_pop = pd.read_csv("population_communes.csv")

df_pop.head()

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658


In [38]:
df_surf = pd.read_csv("surface_departements.csv",
                      encoding = "utf-8")

df_surf.head()

,code_insee,nom,surf_km2
0,974,La Réunion,2505.0
1,11,Aude,6343.0
2,43,Haute-Loire,5003.0
3,13,Bouches-du-Rhône,5247.0
4,47,Lot-et-Garonne,5385.0


### cleaning

In [39]:
df_pop = df_pop.drop(['Code canton', 'Code arrondissement', 'Nom de la commune', 'Code commune'], axis=1)
df_pop = df_pop.groupby(['Code région', 'Code département', 'variable'])['value'].sum().reset_index()

df_pop = df_pop.rename(columns={'Code région': 'code_region',
                                'Code département': 'code_dept'})

df_pop.head()

,code_region,code_dept,variable,value
0,1,971,Population municipale,394110
1,1,971,Population totale,400170
2,2,972,Population municipale,376480
3,2,972,Population totale,382294
4,3,973,Population municipale,269352


In [47]:
df_surf = df_surf.rename(columns={'nom': 'nom_dept'})
                                  
df_surf.head()                            

,code_insee,nom_dept,surf_km2
0,974,La Réunion,2505.0
1,11,Aude,6343.0
2,43,Haute-Loire,5003.0
3,13,Bouches-du-Rhône,5247.0
4,47,Lot-et-Garonne,5385.0


### Pivot

In [41]:
df_pop = df_pop.pivot_table(values='value',
                            columns='variable',
                            index=['code_region', 'code_dept'])

df_pop.head()

,variable,Population municipale,Population totale
code_region,code_dept,,
1,971,394110,400170
2,972,376480,382294
3,973,269352,271829
4,974,852924,862814
11,75,2190327,2210875


In [42]:
df_pop = df_pop.rename(columns={'Population municipale': 'pop_municipale',
                                'Population totale': 'pop_totale'})

df_pop.head()

,variable,pop_municipale,pop_totale
code_region,code_dept,,
1,971,394110,400170
2,972,376480,382294
3,973,269352,271829
4,974,852924,862814
11,75,2190327,2210875


### Create new column

In [43]:
df_pop['pct_pop_municipale'] = df_pop['pop_municipale'] / df_pop['pop_totale'] * 100

df_pop.head()

,variable,pop_municipale,pop_totale,pct_pop_municipale
code_region,code_dept,,,
1,971,394110,400170,98.485644
2,972,376480,382294,98.479181
3,973,269352,271829,99.088765
4,974,852924,862814,98.853751
11,75,2190327,2210875,99.070594


In [44]:
df_pop['part_region'] = df_pop['pop_municipale'] / df_pop.groupby('code_region').transform(sum)['pop_municipale'] * 100

df_pop.head()

,variable,pop_municipale,pop_totale,pct_pop_municipale,part_region
code_region,code_dept,,,,
1,971,394110,400170,98.485644,100.000000
2,972,376480,382294,98.479181,100.000000
3,973,269352,271829,99.088765,100.000000
4,974,852924,862814,98.853751,100.000000
11,75,2190327,2210875,99.070594,18.076282


In [61]:
df_pop = df_pop.rename_axis(None, axis=1).reset_index()
df_pop.head()

,code_region,code_dept,pop_municipale,pop_totale,pct_pop_municipale,part_region
0,1,971,394110,400170,98.485644,100.000000
1,2,972,376480,382294,98.479181,100.000000
2,3,973,269352,271829,99.088765,100.000000
3,4,974,852924,862814,98.853751,100.000000
4,11,75,2190327,2210875,99.070594,18.076282


In [62]:
df_merge = df_pop.merge(right=df_surf, 
                        left_on=['code_dept'],
                        right_on=['code_insee'])

df_merge.head()

,pop_municipale,pop_totale,pct_pop_municipale,part_region,code_insee,nom_dept,surf_km2
0,394110,400170,98.485644,100.000000,971,Guadeloupe,1634.0
1,376480,382294,98.479181,100.000000,972,Martinique,1089.0
2,269352,271829,99.088765,100.000000,973,Guyane,83543.0
3,852924,862814,98.853751,100.000000,974,La Réunion,2505.0
4,2190327,2210875,99.070594,18.076282,75,Paris,105.0


In [63]:
df_merge['densite'] = df_merge['pop_municipale'] / df_merge['surf_km2']

df_merge.head()

,pop_municipale,pop_totale,pct_pop_municipale,part_region,code_insee,nom_dept,surf_km2,densite
0,394110,400170,98.485644,100.000000,971,Guadeloupe,1634.0,241.193390
1,376480,382294,98.479181,100.000000,972,Martinique,1089.0,345.711662
2,269352,271829,99.088765,100.000000,973,Guyane,83543.0,3.224112
3,852924,862814,98.853751,100.000000,974,La Réunion,2505.0,340.488623
4,2190327,2210875,99.070594,18.076282,75,Paris,105.0,20860.257143


In [64]:
df_merge = df_merge.rename(columns={'code_insee': 'code_dept'})
df_merge = df_merge[['nom_dept','code_dept','pop_municipale','pop_totale',
                     'pct_pop_municipale','part_region','densite']]

df_merge.head()

,nom_dept,code_dept,pop_municipale,pop_totale,pct_pop_municipale,part_region,densite
0,Guadeloupe,971,394110,400170,98.485644,100.000000,241.193390
1,Martinique,972,376480,382294,98.479181,100.000000,345.711662
2,Guyane,973,269352,271829,99.088765,100.000000,3.224112
3,La Réunion,974,852924,862814,98.853751,100.000000,340.488623
4,Paris,75,2190327,2210875,99.070594,18.076282,20860.257143


In [68]:
df_merge = df_merge.sort_values(by='code_dept')
df_merge.head()

,nom_dept,code_dept,pop_municipale,pop_totale,pct_pop_municipale,part_region,densite
80,Ain,01,638425,655171,97.444026,8.064089,110.377766
31,Aisne,02,536136,549587,97.552526,8.925380,72.343274
81,Allier,03,339384,349336,97.151167,4.286835,45.993224
92,Alpes-de-Haute-Provence,04,162565,167331,97.151753,3.237103,23.246818
93,Hautes-Alpes,05,141107,146148,96.550757,2.809817,24.768650


In [70]:
df_merge.to_csv('exo_cc_result.csv', index=False)